In [6]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, LSTM, Bidirectional, Dense, 
    Reshape, Dropout, BatchNormalization, Flatten
)
from tensorflow.keras.utils import Sequence
import cv2

# ✅ Paths for Character Dataset
train_char_json = r"C:\Users\Victus\Desktop\major project\json files\character_labels.json"
train_char_images = r"C:\Users\Victus\Desktop\major project\datasets\character"

# ✅ Load Dataset
def load_dataset(json_path, image_folder):
    with open(json_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    images, labels = [], []
    for item in data:
        image_path = os.path.join(image_folder, item["image_filename"])
        if os.path.exists(image_path):
            images.append(image_path)
            labels.append(item["transcription"])
    
    return images, labels

char_images, char_labels = load_dataset(train_char_json, train_char_images)
print(f"✅ Loaded {len(char_images)} character images.")

# ✅ Create Character-to-Index Mapping
def create_char_mapping(labels):
    unique_chars = sorted(set("".join(labels)))
    char_to_index = {char: idx + 1 for idx, char in enumerate(unique_chars)}
    char_to_index["<BLANK>"] = 0  # Ensure blank token for unknown characters
    return char_to_index

char_to_index = create_char_mapping(char_labels)
with open("char_too_index.json", "w", encoding="utf-8") as f:
    json.dump(char_to_index, f, ensure_ascii=False, indent=4)

print(f"✅ Created character mapping with {len(char_to_index)} characters.")

# ✅ Preprocess Character Images (Resized to 100x100)
def preprocess_image(image, target_size=(100, 100)):
    """Ensure image has the correct shape before processing."""
    image = cv2.resize(image, (target_size[1], target_size[0]), interpolation=cv2.INTER_AREA)

    # ✅ Convert to grayscale ONLY if it's not already grayscale
    if len(image.shape) == 3:  # Check if it has 3 channels (RGB/BGR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    image = np.expand_dims(image, axis=-1) / 255.0  # Normalize
    return image

# ✅ Character OCR Data Generator
class CharDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, char_to_index, img_size=(100, 100)):  
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.char_to_index = char_to_index
        self.img_size = img_size
        self.indices = np.arange(len(self.image_paths))
        np.random.shuffle(self.indices)

    def __len__(self):  
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, index):  
        batch_indices = self.indices[index * self.batch_size: (index + 1) * self.batch_size]
        batch_images, batch_labels = [], []

        for i in batch_indices:
            image = cv2.imread(self.image_paths[i], cv2.IMREAD_GRAYSCALE)
            image = preprocess_image(image, self.img_size)
            char_index = self.char_to_index.get(self.labels[i], 0)

            batch_images.append(image)
            batch_labels.append(char_index)

        return np.array(batch_images), np.array(batch_labels)

# ✅ Create Character Data Generator
batch_size = 32
char_train_generator = CharDataGenerator(char_images, char_labels, batch_size, char_to_index)

# ✅ Build Character Recognition Model
def build_char_recognition_model(input_shape, num_classes):
    image_input = Input(shape=input_shape, name="image_input")

    x = Conv2D(64, (3,3), activation="relu", padding="same")(image_input)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dense(num_classes, activation="softmax", name="output")(x)

    return Model(inputs=image_input, outputs=x)

# ✅ Compile & Train Character Model
char_model = build_char_recognition_model((100, 100, 1), len(char_to_index))
char_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
char_model.fit(char_train_generator, epochs=30, verbose=1)
char_model.save("char_recognition_model.keras")

✅ Loaded 72000 character images.
✅ Created character mapping with 35 characters.
Epoch 1/30
   3/2250 ━━━━━━━━━━━━━━━━━━━━ 2:44:09 4s/step - accuracy: 0.0208 - loss: 3.7297    

KeyboardInterrupt: 